# Cyckle Developer Handbook: Basics
This short handbook for developers interested in modifying and contributing to Cyckle-ai provides a fundamental understanding of the source files in Cyckle.
## Chapter 1: ```main.pyx```
This section will go over the concepts described in the ```main.pyx``` source file.
### Generation
This code snippet is how Cyckle generates responses. 

***Developer note: Though we have switched to ```phi-3.5-mini```, we keep ```phi-3``` here since 3.5 is not (officially) supported by ```gpt4all```***

In [ ]:
from gpt4all import GPT4All

system_prompt = '''You are Cyckle, a helpful AI assistant. Your responses should be clear, direct, and relevant to the user's questions. Aim to be informative yet concise.'''
usermodel = GPT4All("Phi-3-mini-4k-instruct.Q4_0.gguf", n_threads=4, allow_download=True)
userinput = "What is the capital of France?"

with usermodel.chat_session(system_prompt=system_prompt):
    reponse = usermodel.generate(userinput, max_tokens=8, temp=0.3, top_k=25, top_p=0.9, repeat_penalty=1.1, n_batch=8)
    print(reponse)


Lets break it down line by line.
#### System Prompt
The system prompt works by telling the AI what it is and how to do it's job. Pretty elementary right?
#### Usermodel
This defines performance parameters and the model being used. Lets look deeper.
#### The model
At the beginning, we define the model with the following:
```python
    GPT4All("Phi-3-mini-4k-instruct.Q4_0.gguf")
```
This tells GPT4All that it must use and initialize the ```Phi3-mini``` AI model. If it does not exist, it will search for it on ```gpt4all.io``` in order to download it.
#### ```n_threads```
In the middle we state the following:
```python
    n_threads=4
```
This tells GPT4All that we want to utilize 4 CPU threads (The amount that would be used if you had a dual-core processor) in order to process the AI model.
#### Download Parameter
Lastly, we state the following:
```python
    allow_download=true
```
This tells GPT4All that it can download data, ```.gguf``` files, and prompt templates from the web.
#### Generation Parameters
The following parameters are used for generating Cyckle's responses
```python
    max_tokens=8, temp=0.3, top_k=25, top_p=0.9, repeat_penalty=1.1, n_batch=8
```
Here's an analysis of each parameter:

```max_tokens``` The maximum length of response in tokens. (Can be changed without worry)

```temp``` The balance between creativity and precision with values between 1 and 0 where 1 is the most creative on the scale and 0 is the most precise on the scale.

```top_k``` The scale between 0 and 50 where lower values have more predictable word outputs and higher values are more random word outputs

```top_p``` Similar to ```top_k``` but could be defined as a parameter for the words it will use. It has values ranging between 0 and 1 with 1 being more wild and a large pool of words or 0 with a more predictable, smaller word pool.

```n_batch``` A lot more technical than the previous parameters, but can be easily described as the amount of tokens being processed at once. Usually for low-spec machines, 4 is a good value but it can be as high as you want until you run out of RAM or VRAM.

```repeat_penalty``` Penalizes the AI for repeating. Typical values are between 1.1 and 1.3 because any higher and the AI will miss out on important repeats.

### Optimization

This code snippet demonstrates how Cyckle optimizes for the user's CPU/GPU. (Adapted for Python instead of Cython)

In [6]:
import psutil
import gpt4all
from gpt4all import GPT4All

def optimize():
    physcores = psutil.cpu_count(logical=False)
    logicores = psutil.cpu_count(logical=True)
    return physcores, logicores

physcores, logicores = optimize()
threads = min(logicores, 8)

GPT4All("Phi-3-mini-4k-instruct.Q4_0.gguf", n_threads=threads)
print(threads)

Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


4


Lets break this down line by line.

#### ```psutil```
```psutil``` is utilized for detecting certain characteristics of a user's system. In this case, it detects the threads in a user's processor.

#### ```physcores```
The amount of physical cores detected by ```psutil```.

#### ```logicores```
The amount of logical cores detected by ```psutil```.

### Reading Tokens
This snippet demonstrates the function of reading tokens from the JSON file. (Adapted to Python from Cython.)

In [ ]:
import json

filename = "data.json"

def read_tokens_from_json():
    try:
        with open(filename, "r") as f:
            data = json.load(f)
            return data.get("tokens", 96)
    except FileNotFoundError:
        print("data.json not found. Falling back to default 256.")
        return 256

modtokens = read_tokens_from_json()

datadict = {
    "tokens": modtokens
}

with open(filename, 'w') as f:
    json.dump(datadict, f, indent=4)
    print("Data written to:", filename)

print("Token amount:", modtokens)


Want to experiment with this? Modify the ```data.json``` file in the root of this directory!

Anyways, let's see what is going on here!

#### Reading the tokens
The following section of our snippet defines how Cyckle reads tokens from ```data.json```.
```python
    def read_tokens_from_json():
    try:
        with open(filename, "r") as f:
            data = json.load(f)
            return data.get("tokens", 96)
    except FileNotFoundError:
        print("data.json not found. Falling back to default 256.")
        return 256
```
This tells Cyckle to load from ```data.json``` expecting the following data structure:
```JSON
{
    "tokens": modtokens
}
```
***Note: If the function fails to find the relevant information or the file ```data.json```, it will fallback to the default number of 256 tokens.***

#### Writing the tokens
The following section of our snippet defines how Cyckle writes tokens to ```data.json```.
```python
    with open(filename, 'w') as f:
    json.dump(datadict, f, indent=4)
    print("Data written to:", filename)
```
This tells Cyckle that the ```datadict``` dictionary (which is formatted to the liking of JSON) should be written into ```data.json```. It also provides extra formatting with the ```indent=4``` option which tells Cyckle to save the JSON with 4 spaces per indentation.

## Chapter 3: ```setup.py```
This section delves into the technical aspects of the ```setup.py``` file used to convert ```.pyx``` into ```.c``` and ```.h``` source files.

```python
from setuptools import setup, Extension
from Cython.Build import cythonize

extensions = [
    Extension(
        name="main",
        sources=["main.pyx"],
        extra_compile_args=["-fPIC"],
    )
]

setup(
    name="CyckleAI",
    ext_modules=cythonize(extensions, language_level="3"),
    zip_safe=False,
)
```

Right now this may seem like technical nonsense, but let's explain and analyze this.

#### Extension Compiler Args
Cython (for some unholy reason) considers this standalone app, an extension. Inside of this, we have a compiler argument titled ```-fPIC```. This argument tells Cython to build this with shared libraries, hence the reason it creates ```main.c``` and ```main.h``` from ```main.pyx```.

#### Language Level
The ```language_level``` parameter defines whether or not you are using Python 2 or 3. Of course, Cyckle isn't ancient and thus does not use Python 3, henceforth ```language_level``` is set to 3.

#### Zip Safe
The ```zip_safe``` parameter defines whether your application should be executed as a "ZipApp" (Basically a normal executable, but in a portable ```.zip```, ```.tar```, etc. format). This is set to false as ZipApps are unsuitable for Cyckle's use case.